In [1]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import math


iris = datasets.load_iris()

In [2]:
# since this is a bunch, create a dataframe and then convert it into a numpy array
iris_df=pd.DataFrame(iris.data)
iris_df['class']=iris.target
iris_df.columns=['sepal_len', 'sepal_wid', 'petal_len', 'petal_wid', 'class']
iris_df.dropna(how="all", inplace=True) # remove any empty lines

#shuffle the data
iris_df = iris_df.sample(frac=1).reset_index(drop=True)

#convert the data into the relevant numpy arrays
irises=iris_df[['sepal_len', 'sepal_wid', 'petal_len', 'petal_wid']].to_numpy()
irises_type=iris_df["class"].to_numpy()

#initialise the weights and biases
k=10

Z1_weights=abs(np.random.randn(4, k)*0.03)
Z1_biases=np.zeros((1, k))
Z2_weights=np.random.randn(k, 3)*0.03
Z2_biases=np.zeros((1, 3))

#print(Z1_weights)
#print(Z1_biases)
#print(Z2_weights)
#print(Z2_biases)

In [3]:
# Split the data into the train and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(irises, irises_type, test_size=0.25, random_state=42)

In [4]:
#Define functions for further use
def softmax(array):
    #defines a softmax function on an array of shape (1, 3)
    sum=0
    for i in array[0]:
        sum+=math.e**i
    for j in range(len(array[0])):
        array[0, j]=math.e**array[0, j]/sum
    return array

def relu(array):
    #defines a relu function on an array of shape (1, k)
    return np.maximum(array, 0)

def loss(output, correct_output):
    #calcusates the Cross-Entropy loss function
    #shape of the output is 1x3, and of the correct output is 1x3
    if isinstance(output, np.ndarray) and output.shape == (1, 3):
        if isinstance(correct_output, np.ndarray) and correct_output.shape == (1, 3):
            #print(output)
            #print(correct_output)
            return -np.dot(np.log(output).reshape(-1), correct_output.reshape(-1))
        else:
            raise TypeError('wrong array shapes')
    else:
        raise TypeError('wrong array shapes')


def single_forward(example_id, X_train, y_train, Z1_weights, Z1_biases, Z2_weights, Z2_biases):
    #Calculates the forward step for a single example using weights&biases as inputs

    #example is a numpy array of shape (4, )
    example=X_train[example_id]

    #define a vector Z1 as weights (k, 4) * example (4,) + biases (10, )
    Z1 = np.dot(example, Z1_weights) + Z1_biases
   # print('Z1: ', Z1)

    # Obtain array of square of each element in x
    A1 = relu(Z1)
  #  print('A1:', A1)

    #define the vector Z2 as weights (3, k) * A1 (k,) + biases (k, )
    Z2 = np.dot(A1, Z2_weights) + Z2_biases
   # print('Z2:', Z2)
    #apply the activation function softmax
    output=softmax(Z2) #A2

    return Z1, A1, Z2, output
     

def forward_prop(X_train, y_train, Z1_weights, Z1_biases, Z2_weights, Z2_biases):
    #This function is meant to calculate the loss function for an example set
    total_loss=0
    total_cases=0
    #for all vectors in TRAINING SET
    for index in range(len(X_train)):
        Z1, A1, Z2, output = single_forward(index, X_train, y_train, Z1_weights, Z1_biases, Z2_weights, Z2_biases)

        correct_class=y_train[index]
        #we need to store the correct class as (1,0,0) or similar np array
        correct_output=np.zeros((1, 3))
        correct_output[0, correct_class]=1

        total_loss+=loss(output, correct_output)
        total_cases+=1
    return total_loss/total_cases



In [6]:
#GRADIENT DESCENT
epochs=600
lr=0.08

#Now, we define a for loop for num_of_steps of gradient descent
for step in range(epochs): #might change to 'while True' later
     #firstly, shuffle the data to avoid learning from the order
     indices = np.random.permutation(len(X_train))

     # Shuffle both arrays using the same indices
     X_train = X_train[indices]
     y_train = y_train[indices]

     #begin with defining arrays for the derivatives
     dl_dW1=np.zeros((4, k))
     dl_dB1=np.zeros((1, k))
     dl_dW2=np.zeros((10, 3))
     dl_dB2=np.zeros((1, 3))



     #sum up the gradients for each example and average them in the end
     for index in range(len(X_train)):
          answer=y_train[index]
          example=X_train[index]
          Z1, A1, Z2, A2 = single_forward(index, X_train, y_train, Z1_weights, Z1_biases, Z2_weights, Z2_biases)
          #Firstly, define the derivative wrt elements of A2
          dl_A2=np.zeros((1, 3))
          dl_A2[0, answer]=-1/A2[0, answer]

          #Now, define dl_dZ2
          dl_dZ2=np.zeros((1,3))
          for j in range(3): #To calculate Z_j
               if j==answer:
                    dl_dZ2[0,j]=A2[0,j]-1
               else:
                    dl_dZ2[0,j]=A2[0,j]
          
          #further, define dl_dA1
          dl_dA1=np.zeros((1, k))
          for i in range(k):
              for j in range(3):
                   dl_dA1[0,i]=dl_dA1[0,i]+dl_dZ2[0,j]*Z2_weights[i, j]
          
          #finally, define dl_dZ1
          dl_dZ1=np.zeros((1, k))
          for i in range(k):
               if Z1[0,i]>0:
                    dl_dZ1[0,i]=dl_dA1[0,i]
               else:
                    dl_dZ1[0,i]=0
          
          #Calculate the derivatives wrt W2&B2 usindg delta_j=dl_dZ_2(j)
          for j in range(3):
               dl_dB2[0,j]=dl_dB2[0,j]+dl_dZ2[0,j]
               for t in range(10):
                    dl_dW2[t, j]=dl_dW2[t, j]+dl_dZ2[0,j]*A1[0,t]
          #Calculate the derivatives wrt W1&B1 delta_j=dl_dZ_1(j)
          for j in range(k):
               dl_dB1[0,j]=dl_dB1[0,j]+dl_dZ1[0,j]
               for t in range(4):
                    dl_dW1[t, j]=dl_dW1[t, j]+dl_dZ1[0,j]*example[t]

     #average over all training examples
     dl_dW1=dl_dW1/len(y_train)
     dl_dB1=dl_dB1/len(y_train)
     dl_dW2=dl_dW2/len(y_train)
     dl_dB2=dl_dB2/len(y_train)

     #redefine the learning rate to align with the magnitude of weights&biases

     #Update the weights & biases
     Z1_weights =  Z1_weights-lr*dl_dW1
     Z1_biases =  Z1_biases-lr*dl_dB1
     Z2_weights = Z2_weights-lr*dl_dW2
     Z2_biases = Z2_biases-lr*dl_dB2

     loss_val = forward_prop(X_train, y_train, Z1_weights, Z1_biases, Z2_weights, Z2_biases)

     print('loss', loss_val)
                    
                    

#define a for loop to do gradient descent
    #calculate first derivatives
    #second derivatives
    #third derivatoves
    #forth derivatives
    #modify the first set of weights and biases
    #modify the second set of weights and biases
    #call the forwad propagation function

loss 1.0760517618808825
loss 1.072369158121537
loss 1.0683837911515144
loss 1.0640564279622777
loss 1.0593475906889547
loss 1.054216512406087
loss 1.048620108287515
loss 1.0425120325883928
loss 1.0358440552079207
loss 1.028569054485753
loss 1.020616930552744
loss 1.0119171920999095
loss 1.0024440465635742
loss 0.9923445030426519
loss 0.9817406457476221
loss 0.9707686727397459
loss 0.9593866572950807
loss 0.9475986558470193
loss 0.9354199997878936
loss 0.9229480428826459
loss 0.9105603376754919
loss 0.8982170088814531
loss 0.8858772427619414
loss 0.8735030348294898
loss 0.8610807565985655
loss 0.8485348585436342
loss 0.8359063049670921
loss 0.8231286311067487
loss 0.8103882352470964
loss 0.7975315071947425
loss 0.7846548170550204
loss 0.771426929738868
loss 0.7578858891806506
loss 0.7442299206747485
loss 0.7301734962579364
loss 0.7160902782282926
loss 0.7018142337743332
loss 0.6875619211593029
loss 0.673260627833913
loss 0.6592172842013645
loss 0.6453857778618843
loss 0.6318773464145885

In [9]:
#Now, we want to verify this on the training set
total_examples=len(y_train)
correct_examples=0
for index in range(len(X_train)):
        Z1, A1, Z2, output = single_forward(index, X_train, y_train, Z1_weights, Z1_biases, Z2_weights, Z2_biases)
        
        prediction=np.argmax(output)
        correct_class=y_train[index]

        if prediction==correct_class:
            correct_examples+=1

print(correct_examples/total_examples)

0.9910714285714286


In [10]:
#Repeat on the test set
total_examples=len(y_test)
correct_examples=0
for index in range(len(X_test)):
        Z1, A1, Z2, output = single_forward(index, X_test, y_test, Z1_weights, Z1_biases, Z2_weights, Z2_biases)
        
        prediction=np.argmax(output)
        correct_class=y_test[index]

        if prediction==correct_class:
            correct_examples+=1

print(correct_examples/total_examples)

0.9473684210526315


In [95]:
# Sid G| UTC — Today at 17:33
# # utility function we will use later when comparing manual gradients to PyTorch gradients
# def cmp(s, dt, t):
#   ex = torch.all(dt == t.grad).item()
#   app = torch.allclose(dt, t.grad)
#   maxdiff = (dt - t.grad).abs().max().item()
#   print(f'{s:15s} | exact: {str(ex):5s} | approximate: {str(app):5s} | maxdiff: {maxdiff}')